In [ ]:
from utils import *
from rf12demo.rf12demo import Rf12Demo
import logging, time
import logging
plogger = logging.getLogger('pymote')
plogger.setLevel(logging.ERROR+1)
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
if "jeelink" in locals() and not jeelink.closed:
    jeelink.close()
jeelink = Rf12Demo(payload_header, payload_body, port='COM3', baudrate=57600, timeout=1)
fileHandler = logging.handlers.RotatingFileHandler('rf12demo.log')
fileHandler.setFormatter(jeelink.loghandler.formatter)
jeelink.logger.addHandler(fileHandler)

In [ ]:
jeelink.set_group(212)

In [ ]:
jeelink.send(0, "SetStatus", {"status": 0})
jeelink.log_received_information(1, log_package_function)


In [ ]:
from pymote import NetworkGenerator

NODES = [1, 2, 3, 4, 6, 7, 9, 10]
#NODES = [6, 7, 9, 10]
#NODES = [1, 2, 3, 4]


net_gen = NetworkGenerator(len(NODES))
net = net_gen.generate_homogeneous_network()
for i, node in enumerate(net.nodes()):
    node.id = NODES[i]
    net.labels[node] = str(node.id)
net.show()


In [ ]:
for node in net.nodes():
    package = dict(("sn%d" % i, 0) for i in range(16))
    package.update(dict(map(lambda (i, n): ("sn%d" % i, n.id), enumerate(net.adj[node]))))
    print node.id, package
    jeelink.send(node.id, "SetNeighbors", package)

In [ ]:
jeelink.send(0, "Reset")
#jeelink.send(NODES, "Reset")

In [ ]:
jeelink.send(9, "Value", {"val": 0})

In [ ]:
from random import random
for node in NODES:
    val = int(random()*len(NODES))
    jeelink.send(node, "Value", {"val": val})
    print node, val
log = jeelink.log_received_information(1, log_package_function)

In [ ]:
log = jeelink.log_received_information(1, log_package_function)

In [ ]:
jeelink.send(0, "Sync")
log = jeelink.log_received_information(1, log_package_function)

In [ ]:
log = jeelink.log_received_information(1, log_package_function)

In [ ]:
from time import sleep
for i in range(10):
    sleep(0.36)
    jeelink.send(0, "Sync")
log = jeelink.log_received_information(1, log_package_function)

In [ ]:
jeelink.send(10, "SetNeighbors", {'sn10': 0, 'sn11': 0, 'sn12': 0, 'sn13': 0, 'sn14': 0, 'sn15': 0, 'sn6': 0, 'sn7': 0, 'sn4': 0, 'sn5': 0, 'sn2': 0, 'sn3': 0, 'sn0': 2, 'sn1': 1, 'sn8': 0, 'sn9': 0})

### Generating hypercube topology (not in use)

In [ ]:
maxnode = 9
for i in range(maxnode+1):
    #print i, "=>",
    for k in range(4):
        neighbor = i^(1<<k)
        if neighbor and neighbor<=maxnode and i:
            print i, neighbor, ",",
            #print "==", i^neighbor

In [ ]:
from IPython.display import Image
Image("hypercube.png")